In [1]:
%%capture

!pip install -q transformers

!pip install sentencepiece
!pip install git+https://github.com/csebuetnlp/normalizer

! pip install bangla==0.0.2
# !pip install num2words
!nvidia-smi

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import re
# from num2words import num2words
import os
from tqdm.auto import tqdm
tqdm.pandas()

import transformers

import torch
import bangla
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,pipeline
from normalizer import normalize # pip install git+https://github.com/csebuetnlp/normalizer

import warnings
warnings.filterwarnings("ignore")

# from pandarallel import pandarallel
# pandarallel.initialize(progress_bar=True,nb_workers=8)

os.environ["TOKENIZERS_PARALLELISM"] = "true"

print(torch.__version__)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

1.11.0


In [3]:
seerah = pd.read_csv('../input/yasir-qadhi-seerah/seerah.csv')
seerah.head()

,id,url,title,text,transcription
0,1,https://arqadhi.blogspot.com/2015/11/001-speci...,001 - Specialties of Prophet Muhammad SAW,"Insha'Allah (إن شاء الله - God willing), befor...",[Transcribed by Br. Safwan Khan & Faizan]\nsaf...
1,2,https://arqadhi.blogspot.com/2015/11/002-speci...,002 - Specialties of Prophet Muhammad SAW (Par...,In our last halaqa (حلقة) we mentioned only ar...,"[Transcribed by Br. Safwan Khan, Faizan, Zara ..."
2,3,https://arqadhi.blogspot.com/2015/11/003-why-s...,003 - Why Study the Seerah and Pre-Islamic Arabia,Before we discuss the life of the Prophet PBUH...,"[Transcribed by Br. Safwan Khan, Faizan & Zohr..."
3,4,https://arqadhi.blogspot.com/2015/11/004-relig...,004 - Religious Status of the World Before Islam,Today we will shed some light upon the religio...,"[Transcribed by Br. Safwan Khan, Faizan, Khali..."
4,5,https://arqadhi.blogspot.com/2015/11/005-genea...,005 - Genealogy & Year of the Elephant,"Insha'Allah ta'ala, today will be, as we said,...",[Transcribed[DOWNLOAD MP3]\n\nInsha'Allah ta'a...


In [4]:
print(torch.__version__)
transformers.__version__

1.11.0


'4.20.1'

In [5]:
%%time

def text_cleaner(text):
    res = re.sub('\'','',text)
    res = re.sub('\n','.',res)
    
    res = re.sub('﴾','',res)
    res = re.sub('﴿','',res)
    
    res = re.sub('«','',res)
    res = re.sub('»','',res)
    
#     res = res.replace('Prev.Next','')
    return res
    
seerah["text"]=seerah.text.progress_apply(lambda text: text_cleaner(text))

  0%|          | 0/101 [00:00<?, ?it/s]

CPU times: user 52.5 ms, sys: 3.52 ms, total: 56 ms
Wall time: 56.7 ms


In [6]:
def tag_arabic_text(text,ar_pattern=u'[\u0600-\u06FF]+',english_only = False):
    # remove multiple spaces
    data=re.sub(' +', ' ',text)
    texts=[]
    if "।" in data:punct="।"
    elif "." in data:punct="."
    else:punct="\n"
    for text in data.split(punct):    
        # create start and end
        text="start"+text+"end"
        # tag text
        parts=re.split(ar_pattern, text)
        parts=[p for p in parts if p.strip()]
        parts=set(parts)
        for m in parts:
            if len(m.strip())>1:text=text.replace(m,f"</ar>{m}<ar>")
        # clean-up invalid combos
        text=text.replace("</ar>start",'')
        text=text.replace("end<ar>",'')
        texts.append(text)
    text=f"{punct}".join(texts)
    if(english_only):
        #https://stackoverflow.com/questions/55656429/replace-or-remove-html-tag-content-python-regex
        return re.sub(r'(?s)<ar>.*?</ar>', '', text)
    return text

In [7]:
%%time

seerah["text"]=seerah.text.progress_apply(lambda seerah_eng: tag_arabic_text(seerah_eng,english_only=False))

  0%|          | 0/101 [00:00<?, ?it/s]

CPU times: user 609 ms, sys: 8.57 ms, total: 617 ms
Wall time: 649 ms


In [8]:
# seerah["text"][0]

In [9]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(torch_device)
model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/banglat5_nmt_en_bn").to(torch_device)
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglat5_nmt_en_bn",use_fast=True)

def translate_en_bn(input_sentence):
    input_ids = tokenizer(normalize(input_sentence), return_tensors="pt").input_ids
    input_ids = input_ids.to(torch_device)
    generated_tokens = model.generate(input_ids)
    decoded_tokens = tokenizer.batch_decode(generated_tokens)[0]
    decoded_tokens=decoded_tokens.replace("<pad>","").replace("</s>","")
    sen=decoded_tokens.split()
    words=[w for w in sen if w.strip()]
    sen=" ".join(words)
    return decoded_tokens

text=translate_en_bn("alhamdulillah for everything.")
print(text)

cuda


Downloading:   0%|          | 0.00/766 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/945M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

 সবকিছুর জন্য আলহামদুলিল্লাহ।


In [10]:
en_text = '''In the Name of Allah, the Most Gracious, the Most Merciful. <ar> ﴿الْحَمْدُ للَّهِ رَبِّ الْعَـلَمِينَ﴾ </ar>
(Allah, the Exalted, said, `I have divided the prayer (Al-Fatihah) into two halves between 
Myself and My servant, and My servant shall have what he asks for.
If he says,' <ar> بِسْمِ اللَّهِ الرَّحْمَـنِ الرَّحِيمِ </ar> Allah says, `My servant has praised Me.' '''

def EN_AR_to_BN_AR_Translator(en_text,tag_text = False):
    '''
    translates multilingual english-arabic code mixed text into 
    multilingual bengali-arabic code mixed text
    ''' 
    if(tag_text):
        en_text = tag_arabic_text(en_text,english_only=False)
    
    sentenceEnders = re.compile('[.!?]')
    sentences = sentenceEnders.split(en_text)
    main_list = []
    for i in range(len(sentences)):
        
        list_str = sentences[i].split('<ar>')
        if(len(list_str) == 1):
            main_list.append(list_str[0])
        else:
            for j in range(len(list_str)):
                if('</ar>' in list_str[j]):
                    list_str1 = list_str[j].split('</ar>')
                    main_list.append("<ar>"+list_str1[0]+"</ar>")
                    main_list.append(list_str1[1])
                else:
                    main_list.append(list_str[j])

    while(" " in main_list):
        main_list.remove(" ")
    for idx in range(len(main_list)):
        if('<ar>' not in main_list[idx] or '</ar>' not in main_list[idx]):
            
            output_sentence = []
            for word in main_list[idx].split():
                output_sentence.append(word)
     
            main_list[idx] = ' '.join(output_sentence)
            #numerizer
            main_list[idx] = bangla.convert_english_digit_to_bangla_digit(main_list[idx])
            # multilingual english-arabic to multilingual bengali-arabic
            try:
                if len(main_list[idx])>1:
                    main_list[idx]=translate_en_bn(main_list[idx])
                            
            except:
                print("failed -> ",main_list[idx])
    
    bn_mlt = " ".join(main_list)
    bn_mlt = re.sub(' ্ ','',bn_mlt)
    bn_mlt = re.sub("\\'","",bn_mlt)#replace \'
    bn_mlt = re.sub('<unk>','',bn_mlt)
    
    bn_mlt = bn_mlt.replace("<ar>"," ")
    bn_mlt = bn_mlt.replace("</ar>"," ")
    return bn_mlt
        
EN_AR_to_BN_AR_Translator(en_text,tag_text = False)

' আল্লাহ তাআলার নামে, যিনি পরম দয়ালু, পরম দয়ালু   ﴿الْحَمْدُ للَّهِ رَبِّ الْعَـلَمِينَ﴾    (আল্লাহ, যিনি মহান, বললেন, আমি আমার ও আমার বান্দার মধ্যে নামাযকে (আল-ফাতিহা) দুই ভাগে বিভক্ত করে দিয়েছি। আমার বান্দা যা চাইবে, তা-ই পাবে।  যদি সে বলে,   بِسْمِ اللَّهِ الرَّحْمَـنِ الرَّحِيمِ    আল্লাহ বলেন, আমার বান্দা আমার প্রশংসা করেছে '

In [11]:
# %%time

# seerah["seerah_bn"] = seerah["text"]
# for i in range(1):
#     seerah["seerah_bn"][i] = EN_AR_to_BN_AR_Translator(seerah.text[i],tag_text = False)

In [12]:
%%time
seerah["seerah_bn"]=seerah.text.progress_apply(lambda en_text: EN_AR_to_BN_AR_Translator(en_text))

  0%|          | 0/101 [00:00<?, ?it/s]

CPU times: user 3h 25min 9s, sys: 13.7 s, total: 3h 25min 23s
Wall time: 3h 26min 2s


In [13]:
seerah.to_csv('./en_bn_seerah.csv',index = False)

In [14]:
seerah

,id,url,title,text,transcription,seerah_bn
0,1,https://arqadhi.blogspot.com/2015/11/001-speci...,001 - Specialties of Prophet Muhammad SAW,InshaAllah (<ar>إن شاء الله</ar> - God willing...,[Transcribed by Br. Safwan Khan & Faizan]\nsaf...,ইনশাআল্লাহ ( إن شاء الله সিরাহ্র যাত্রা শু...
1,2,https://arqadhi.blogspot.com/2015/11/002-speci...,002 - Specialties of Prophet Muhammad SAW (Par...,In our last halaqa (<ar>حلقة</ar>) we mentione...,"[Transcribed by Br. Safwan Khan, Faizan, Zara ...",আমাদের শেষ হালাকাতে ( حلقة আমরা মাত্র ১৫ জ...
2,3,https://arqadhi.blogspot.com/2015/11/003-why-s...,003 - Why Study the Seerah and Pre-Islamic Arabia,Before we discuss the life of the Prophet PBUH...,"[Transcribed by Br. Safwan Khan, Faizan & Zohr...",মহানবী হযরত মুহাম্মদ (সা.) এর জীবনী আলোচনা কর...
3,4,https://arqadhi.blogspot.com/2015/11/004-relig...,004 - Religious Status of the World Before Islam,Today we will shed some light upon the religio...,"[Transcribed by Br. Safwan Khan, Faizan, Khali...",আজ আমরা নবী পিবিইউ এর আগমনের আগে আরব এবং বিশ্...
4,5,https://arqadhi.blogspot.com/2015/11/005-genea...,005 - Genealogy & Year of the Elephant,"InshaAllah taala, today will be, as we said, a...",[Transcribed[DOWNLOAD MP3]\n\nInsha'Allah ta'a...,"ইনশাআল্লাহ তালা, আমরা যেমন বলেছি, আজ সিরাহর প..."
...,...,...,...,...,...,...
96,97,https://arqadhi.blogspot.com/2015/11/097-maria...,097 - Maria the Copt & Death of Ibrahim,Today and the next lesson we will catch up wit...,[Transcribed by Br. Safwan Khan & Faizan]\nsaf...,আজ এবং পরের পাঠে আমরা নবী পিবিইউএইচ এর জীবনের...
97,98,https://arqadhi.blogspot.com/2015/11/098-marit...,098 - Marital Dispute With the Wives,We will resume from where we left off. We disc...,[Transcribed by Br. Safwan Khan & Faizan]\nsaf...,আমরা যেখান থেকে শুরু করেছিলাম সেখান থেকে আবার...
98,99,https://arqadhi.blogspot.com/2015/10/099-verse...,099 - Verse of Sword & Hajj of Abu Bakr,We are winding up with the seerah now. Maybe t...,[Transcribed by Br. Safwan Khan & Faizan]\nsaf...,আমরা এখন সিরাহর সাথে শেষ করছি এর পর হয়তো আর...
99,100,https://arqadhi.blogspot.com/2015/10/100-farew...,100 - The Farewell Hajj,Today we will discuss the incident of the Hajj...,[Transcribed by Br. Safwan Khan & Faizan]\nsaf...,আজ আমরা হাজ্জাত আল ওয়াদার ঘটনা নিয়ে আলোচনা ...
